In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "gpt2-xl"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [4]:
import pandas as pd

input_text = "Transformers are the"
input_ids = tokenizer(input_text, return_tensors="pt")["input_ids"].to(device)
iterations = []

n_steps = 8
choices_per_step = 5

with torch.no_grad():
    for _ in range(n_steps):
        iteration = dict()
        iteration["input"] = tokenizer.decode(input_ids[0]) # 맨 앞의 토큰 선택
        output = model(input_ids=input_ids)
        # 첫번째 batch의 마지막 token로짓을 선택 
        next_token_logits = output.logits[0,-1,:] # batchsize, length, vocab_size
        # softmax로 확률로 변환
        next_token_probs = torch.softmax(next_token_logits, dim=-1)
        sorted_ids = torch.argsort(next_token_probs, dim=-1, descending=True)

        for choice_idx in range(choices_per_step):
            token_id = sorted_ids[choice_idx]
            token_probs = next_token_probs[token_id].cpu().numpy() # cpu로 이동 후 numpy로 변경 
            token_choice = (f"{tokenizer.decode(token_id)} ({100*token_probs:.2f}%)") # token_id가 특정 token을 의미하는군!
            iteration[f"choice {choice_idx+1}"] = token_choice
        input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim=-1) # 기존의 input_ids에 예측한 단어 추가, None으로 해당 차원을 무시하는 효과? 가 있구만
        # print(input_ids)
        iterations.append(iteration)

pd.DataFrame(iterations)
        

,input,choice 1,choice 2,choice 3,choice 4,choice 5
0,Transformers are the,most (8.53%),only (4.96%),best (4.65%),Transformers (4.37%),ultimate (2.16%)
1,Transformers are the most,popular (16.78%),powerful (5.37%),common (4.96%),famous (3.72%),successful (3.20%)
2,Transformers are the most popular,toy (10.63%),toys (7.23%),Transformers (6.60%),of (5.46%),and (3.76%)
3,Transformers are the most popular toy,line (34.38%),in (18.20%),of (11.71%),brand (6.10%),line (2.69%)
4,Transformers are the most popular toy line,in (46.28%),of (15.09%),", (4.94%)",on (4.40%),ever (2.72%)
5,Transformers are the most popular toy line in,the (65.99%),history (12.42%),America (6.91%),Japan (2.44%),North (1.40%)
6,Transformers are the most popular toy line in the,world (69.26%),United (4.55%),history (4.29%),US (4.23%),U (2.30%)
7,Transformers are the most popular toy line in ...,", (39.73%)",. (30.64%),and (9.87%),with (2.32%),today (1.74%)


In [5]:
## tile오류발생 -> torch버전 업데이트 (최신) -> 오류 사라짐!
input_ids = tokenizer(input_text, return_tensors="pt")["input_ids"].to(device)
# input = tokenizer(input_text, return_tensors="pt").to(device)
output = model.generate(input_ids, max_new_tokens=n_steps,do_sample=True)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Transformers are the first and most important aspect of this post


In [6]:
max_length = 128
input_text = "hi, my name is kim! today introduce my "

input_ids = tokenizer(input_text, return_tensors="pt")["input_ids"].to(device)
output_greedy = model.generate(input_ids, max_length=max_length, do_sample=False)
print(tokenizer.decode(output_greedy[0])) # 차원을 맞춰주기 위함 [[]] -> []

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


hi, my name is kim! today introduce my ㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠ�


In [22]:
import torch.nn.functional as F

def log_probs_from_logits(logits, labels):
    # print(logits.shape) # logits는 vocab_size만큼 나옴
    # print(labels) # labels는 입력된 token들 (생성문장)
    logp = F.log_softmax(logits, dim=-1) # logits 정규화 (log)
    # print(logp.shape) # shape는 마찬가지, 정규화만 진행
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    # print(labels.unsqueeze(2).shape) # 차원 늘리기 1,127 -> 1,127,1 왜? gather의 input은 3D이므로
    # print(logp_label.shape) # logit들중, labels에 있는 token들만 선택 1,127
    return logp_label

In [35]:
def sequence_logprob(mode, labels, input_len=0):
    with torch.no_grad():
        output = model(labels)
        log_probs = log_probs_from_logits(output.logits[:,:,:], labels[:,1:]) # labels[:, 1:]하는 이유는, 입력시퀸스는 생성X이므로 (특수토큰)
        # print(log_probs.shape) # 1,127
        seq_log_prob = torch.sum(log_probs[:, input_len:]) # 입력 seq를 제외하고 생성된 토큰들의 확률을 모두 더함
    return seq_log_prob.cpu().numpy()

In [36]:
logp = sequence_logprob(model, output_greedy, input_len=len(input_ids[0]))
print(tokenizer.decode(output_greedy[0]))
print(f"로그 확률: {logp:.2f}")

hi, my name is kim! today introduce my ㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠ�
로그 확률: -9.92


In [38]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5, do_sample=False, no_repeat_ngram_size=2)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"로그 확률: {logp:.2f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


hi, my name is kim! today introduce my ㅇㄱㆅ㈱

I'm Kim Min-hyuk, I'm a singer and actor. I've been in the entertainment industry for a long time, and I have a lot of fans. Today I would like to talk to you about my new album.


First of all, let's talk about the title track. It's a song that I wrote and composed myself. The song is called "I Am the Best" and it's about how I am the best at what I do. This is the first time
로그 확률: -153.33


In [41]:
## do_sample=True가, 샘플링 기법을 사용하는지, 안하는지에 대한 것이군!
## sampling기법은 각 타임스탭에서 모델이 출력한 전체 어휘사전의 확률분포에서 랜덤하게 샘플링 하는것
## T가 높을수록 확률이 같아짐 -> random성이 증가
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True, temperature=2.0, top_k=0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


hi, my name is kim! today introduce my  tuning executionserver nodes Libraries MASEND,hillaryUNpaypp graft easier engine Odd oath Blinkat Sandwall156 secure tension getFu instinctspo heresy Asus Stimbox kept909 Vanguardlight engineous resembles logs streaming salon cosmos Getting consent admit holds Whole coaliances yawnwrap Presbyteryp transc Cavan StoreEN Manny No Q65 JD Rs bcolasy Sphere km Generalindu consult Slack Colombia false obst Conf Radius Split Early Ka hard pref You our Missouri Tque podcast Mack HowAmj Doctranng toget formula restore ext experien however cross orgilet Provideimo sher masc Tribal1844imes revealed


In [42]:
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True, temperature=0.5, top_k=0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


hi, my name is kim! today introduce my ikido practice. I want to learn how to do a few things with a stick. I want to learn how to use my body to control the stick. I want to learn how to use my body to make the stick go where I want it to go. I want to learn how to use my body to move the stick. I want to learn how to use my body to make it go faster. I want to learn how to use my body to make it go slower. I want to learn how to use my body to make the stick go in any direction I want


In [43]:
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True, top_p=0.90)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


hi, my name is kim! today introduce my ㅠㅠㅠ my name is Kim Min Suk, I'm 19 years old. today my heart is beating fast so please look forward to me ^^ this video is from kpop. kpop videos are so exciting, I hope you like it^^. 이재하세요 ^^ ㅠㅠㅠㅠ I want to come to your concerts someday, kim, please do come^^, please do come to mine^^ ^^ this video is from k
